In [1]:
import math
from sortedcontainers import SortedDict
import random
import numpy 
import matplotlib.pyplot as plt

In [2]:
# Global Parameters:
t_max = 12*30 #352 months in hrs (assume no nights so 12 hr = 1 day)
timeline = SortedDict() #New and improved timeline (using a sorted dictionary)!
FV_param = [0, t_max] #5days, tmax/3 #0.2 #males/0.19 #totally random
FG_param = 3 #larger values lead to foraging ending earlier, because taus are smaller
SB_param = 1.5 #larger values lead to bowerstay ending earlier, because taus are smaller
RB_param = 6 #made up -- takes 30mins (every time -- no distribution) to repair bower
MA_param = 0.1 #made up -- takes 6mins (every time -- no distribution) to maraud bower
males = 100
next_forage = numpy.random.normal(loc=40000000, scale=.167, size=males)
bower_states = [0] * males
male_states = [0] * males 
strategy_states=[0]*males
mating_states = [0] * males
fitness_states = numpy.array([0] * males)
success_rate = 1
success_times = [] #going to change this to be a vector of times when the female leaves (at this point, sucessful mating has occured)
recents_list = []
max_visits = 6 #STEEEEVEEEE
bird_speed = 12 * 3600 #m/hr (12m/s)
x_dim, y_dim = 400, 500 #dimensions of environment
FperM = 3 #number of sexually mature females per sexually mature males in a bowerbird population
num_sims = 1
not_maraud_time = 0.15

new_tic_glob={}

d={} 

for i in range(males):
    d["next_forages{0}".format(i)]=[]
    d["forage_times{0}".format(i)]=[]

In [3]:
def ticketgenerator(t, tau, ow, ac, targ):
    ticket = {
        'time': t, #time at which the activity starts
        'tau': tau, #how long the activity lasts
        'owner': ow, #who is doing the activity (could be a female if action=-3)
        'action': ac, #which activity (see key)
        'target': targ, #target of the activity (owner=target unless action=-3,-4,-5, or -6)
    }
    return ticket

In [4]:
#Adds tickets to timeline- sorts by time
def addtotimeline(ticket):
    global timeline
    timeline[(ticket["time"], ticket["owner"])] = ticket

In [5]:
# creation of the environment

def envgenerator(males, x_dim, y_dim):
    Xs = numpy.random.rand(males) * x_dim
    Ys = numpy.random.rand(males) * y_dim
    return [Xs,Ys]

def dists(xy, bird_speed):
    male_dist = numpy.zeros(males * males).reshape((males, males))
    travel_times = numpy.zeros(males * males).reshape((males, males))
    for i in range(males):
        for j in range(i + 1, males):
            dist = math.sqrt((xy[0][j] - xy[0][i]) ** 2 + (xy[1][j] - xy[1][i]) ** 2)
            travel = dist / bird_speed
            male_dist[j][i] = dist
            male_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return [male_dist,travel_times]

# the probability of choosing a neighbor at distance x is proportional to exp(-\lambda x)
# choose lambda such that 99% of the mass is before 800 meters
improb = 0.99
improb_distance = 800
lamb = -math.log(1 - improb) / improb_distance

#will write female preference based on cumulative exponential decay (lambda=.00576)
def preferences(male_dist, males, lamb):
    # compute exponential of each coefficient
    visit_preferences = numpy.exp(-lamb * male_dist)
    # remove the identity matrix (exp(0) = 1)
    visit_preferences = visit_preferences - numpy.eye(males)
    # make rows sum to one
    visit_preferences = (visit_preferences.transpose() / numpy.sum(visit_preferences, 1)).transpose()
    return visit_preferences

In [6]:
def nextFGtaugenerator(): #This generates the time until next_forage
    interval=-1
    while interval<=0: #ensures we pick positive values until next_forage
        interval=numpy.random.normal(loc=.4, scale=.167, size=1)[0]
    return interval
    

# function for determining the next time based on our rate parameters
def nexttau(action, ow, targ): 
    new_tau=-1
    while new_tau<=0: #ensures we pick positive values for new_tau
        switcher = { 
            -1: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0], #choose when to leave bower (generate a tau for bower stay)
            -2: numpy.random.gamma(shape=1.5, scale=5, size=1)[0]/60, #choose when to stop foraging (generate a tau for foraging)
            -3: numpy.random.uniform(FV_param[0], FV_param[1]), #FV_param... totally arbitrary so we should think about it
            -4: travel_times[ow][targ],
            -5: MA_param, #in the future we'll do something with it
            -6: travel_times[targ][ow],
            -7: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0],
        }
        new_tau=switcher.get(action)
    return new_tau

def futurebuilder(old_tic, new_ac, new_targ): #returns new ticket after ensuring it doesn't exceed t_max
    old_t=old_tic['time']
    new_tic={}
    if math.isclose(old_t, t_max, rel_tol=0, abs_tol=.0000001)==0: #if old_t is not essentially t_max
        ow=old_tic['owner']
        new_tau=nexttau(new_ac, ow, new_targ)
        end_t=old_t+new_tau #this time is the time when the action has ended
        if end_t>t_max: 
            new_tau=new_tau-(end_t-t_max) #modify the new_tau so end_t will be t_max
            end_t=old_t+new_tau #recalculate end_t
        new_tic=ticketgenerator(end_t, new_tau, ow, new_ac, new_targ) #now the time represents the end time for the action
    return new_tic

In [7]:
def statechanger(new_tic): #calls functions to change states of males i.e. for bower destruction, absence from bower
    options = {-1: SBstatechanger,
               -2: FGstatechanger,
               -3: FVstatechanger,
               -4: MTstatechanger,
               -5: MAstatechanger, 
               -6: MRstatenotchanger,
               -7: RBstatechanger
              }
    options[new_tic['action']](new_tic)
    
def SBstatechanger(new_tic): 
    ow=new_tic['owner']
    t=new_tic['time']
    male_states[ow]=t #male is at his bower / store time so that we FV can access and modify SB_tic in sorted dict

def FGstatechanger(new_tic):
    ow=new_tic['owner']
    male_states[ow]=0 #Male is now absent from bower

def FVstatechanger(new_tic): #here the t is the start time... idea for future: female start and end tics?
    targ=new_tic['target']
    t=new_tic['time']
    if bower_states[targ]==0 and male_states[targ]!=0 and mating_states[targ]<t:
        fitness_states[targ]+=1
        mating_end=.25+t #can use dist instead of .25
        if t<mating_end:
            SB_tic=tl_dict[t] #PSEUDO
            SB_tic['tau']=SB_tic['tau']+(mating_end-SB_tic['time'])
            SB_tic['time']=mating_end
            #re-add tic to sorted dict (if not done automatically)
        mating_states[targ]=mating_end      
    
def MTstatechanger(new_tic):
    ow=new_tic['owner']
    male_states[ow]=0  #Male is now absent from bower

def MAstatechanger(new_tic): #store number of successful marauds of each male --> MAKE A NEW STATE
    targ=new_tic['target']
    if bower_states[targ]==0 and male_states[targ]==0:
        bower_states[targ]=-1*RB_param
    #MAKE IT EITHER TAKE maraud_time or 0 (in the else)
    
def MRstatenotchanger(new_tic):
    pass

def RBstatechanger(new_tic):
    tau=new_tic['tau']
    if bower_states[ow]+tau>0:
        new_tic['time']-=bower_states[ow]+tau #think about it more in the AM :)
        tau=-1*bower_states[ow]
        new_tic['tau']=tau
        #add changes to timeline dict if necessary
    bower_states[ow]+=tau
    male_states[ow]=new_tic['time'] #although it doesn't matter from an FV POV
    

#choose what action / sequence of actions to do next
def decider(tic):
    ow = tic['owner']
    ac = tic['action'] 
    t = tic['time']
    targ=ow #will be reset in the case of -4
    if t>next_forage[ow]:
        d["forage_times{0}".format(ow)].append(t)
        #print("t is greater than next_FG[{:f}]: t={:f}, next_FG={:f}".format(ow, t, next_forage[ow]))
        new_ac=-2 #forage
        d["next_forages{0}".format(ow)].append(next_forage[ow])
        next_forage[ow] = t+nextFGtaugenerator() #forage 30 times / day on average (but underestimates)
    elif bower_states[ow]<0:
        new_ac=-7 #repair bower
    else:
        if random.random()<strategy_states[ow]: #if male chooses to maraud 
            targ=numpy.random.choice(list(range(males)), p=visit_preferences[ow])
            new_ac=-4 #maurad travel
        else: #if male chooses to stay at bower
            new_ac=-1 #stay at bower
    new_tic=futurebuilder(tic, new_ac, targ) #ONE IDEA -- separate ticket generation and adding to timeline
    return new_tic

In [8]:
xy = envgenerator(males, x_dim, y_dim)

travel_mats=dists(xy, bird_speed)
male_dist=travel_mats[0]
travel_times=travel_mats[1]

visit_preferences=preferences(male_dist, males, lamb)

In [9]:
for i in range(males):
    addtotimeline(ticketgenerator(0.0, 0.0, i, -2, i))


print(timeline)
print(next_forage)

SortedDict({(0.0, 0): {'time': 0.0, 'tau': 0.0, 'owner': 0, 'action': -2, 'target': 0}, (0.0, 1): {'time': 0.0, 'tau': 0.0, 'owner': 1, 'action': -2, 'target': 1}, (0.0, 2): {'time': 0.0, 'tau': 0.0, 'owner': 2, 'action': -2, 'target': 2}, (0.0, 3): {'time': 0.0, 'tau': 0.0, 'owner': 3, 'action': -2, 'target': 3}, (0.0, 4): {'time': 0.0, 'tau': 0.0, 'owner': 4, 'action': -2, 'target': 4}, (0.0, 5): {'time': 0.0, 'tau': 0.0, 'owner': 5, 'action': -2, 'target': 5}, (0.0, 6): {'time': 0.0, 'tau': 0.0, 'owner': 6, 'action': -2, 'target': 6}, (0.0, 7): {'time': 0.0, 'tau': 0.0, 'owner': 7, 'action': -2, 'target': 7}, (0.0, 8): {'time': 0.0, 'tau': 0.0, 'owner': 8, 'action': -2, 'target': 8}, (0.0, 9): {'time': 0.0, 'tau': 0.0, 'owner': 9, 'action': -2, 'target': 9}, (0.0, 10): {'time': 0.0, 'tau': 0.0, 'owner': 10, 'action': -2, 'target': 10}, (0.0, 11): {'time': 0.0, 'tau': 0.0, 'owner': 11, 'action': -2, 'target': 11}, (0.0, 12): {'time': 0.0, 'tau': 0.0, 'owner': 12, 'action': -2, 'targe

In [10]:
acc=len(timeline)
new_tic=ticketgenerator(0.0, 0.0, 0, -2, 0) #for testing
for key in timeline:
    acc+=1
    print("enter loop {:d}".format(acc))
    tic=timeline[key]
    print(new_tic)
    print(tic)
    #if tic!=new_tic:
        #print("error^")
        #print(tic)
        #print(new_tic)
    #print("this")
    #print(tic)
    ac=tic['action']
    ow=tic['owner']
    targ=tic['target']
    if ac==-4 or ac==-5:
        #generate a -5 or -6 tic as new_tic and pass it in
        new_tic=futurebuilder(tic, ac-1, targ)
    #elif ac==-3:
    #    #d["rl{0}".format(ow)]=FVtickethandler(next_tic, d["rl{0}".format(ow)])
    #    new_tic=FVtickethandler()
    else:
        new_tic=decider(tic)
    if new_tic=={}:
        print("done (for some owner)!")
    else:
        statechanger(new_tic)
        #print("produces this:")
        #print(new_tic)
        addtotimeline(new_tic)

enter loop 101
{'time': 0.0, 'tau': 0.0, 'owner': 0, 'action': -2, 'target': 0}
{'time': 0.0, 'tau': 0.0, 'owner': 0, 'action': -2, 'target': 0}
enter loop 102
{'time': 0.1612699215238316, 'tau': 0.1612699215238316, 'owner': 0, 'action': -1, 'target': 0}
{'time': 0.0, 'tau': 0.0, 'owner': 1, 'action': -2, 'target': 1}
enter loop 103
{'time': 0.15881108296962151, 'tau': 0.15881108296962151, 'owner': 1, 'action': -1, 'target': 1}
{'time': 0.0, 'tau': 0.0, 'owner': 2, 'action': -2, 'target': 2}
enter loop 104
{'time': 0.2455580805349827, 'tau': 0.2455580805349827, 'owner': 2, 'action': -1, 'target': 2}
{'time': 0.0, 'tau': 0.0, 'owner': 3, 'action': -2, 'target': 3}
enter loop 105
{'time': 0.31559025965831466, 'tau': 0.31559025965831466, 'owner': 3, 'action': -1, 'target': 3}
{'time': 0.0, 'tau': 0.0, 'owner': 4, 'action': -2, 'target': 4}
enter loop 106
{'time': 0.2117874476399128, 'tau': 0.2117874476399128, 'owner': 4, 'action': -1, 'target': 4}
{'time': 0.0, 'tau': 0.0, 'owner': 5, 'ac

{'time': 0.7133143945255505, 'tau': 0.20031707251813258, 'owner': 63, 'action': -1, 'target': 63}
enter loop 579
{'time': 0.8778985609240139, 'tau': 0.16458416639846338, 'owner': 63, 'action': -1, 'target': 63}
{'time': 0.7137103189547918, 'tau': 0.2795358127078639, 'owner': 25, 'action': -1, 'target': 25}
enter loop 580
{'time': 0.8973709797684561, 'tau': 0.1836606608136644, 'owner': 25, 'action': -1, 'target': 25}
{'time': 0.7142481926494126, 'tau': 0.27215998298618055, 'owner': 32, 'action': -1, 'target': 32}
enter loop 581
{'time': 0.9042014828193894, 'tau': 0.18995329016997675, 'owner': 32, 'action': -1, 'target': 32}
{'time': 0.7147007348562271, 'tau': 0.2486218036699645, 'owner': 59, 'action': -1, 'target': 59}
enter loop 582
{'time': 0.8209109745026648, 'tau': 0.10621023964643767, 'owner': 59, 'action': -1, 'target': 59}
{'time': 0.7153867506741425, 'tau': 0.07344019447709167, 'owner': 18, 'action': -1, 'target': 18}
enter loop 583
{'time': 0.8916602364225168, 'tau': 0.17627348

{'time': 1.6113245885087477, 'tau': 0.10636824951418214, 'owner': 18, 'action': -1, 'target': 18}
{'time': 1.506356213725451, 'tau': 0.12358336375816921, 'owner': 58, 'action': -1, 'target': 58}
enter loop 1049
{'time': 1.8973252678919175, 'tau': 0.39096905416646655, 'owner': 58, 'action': -1, 'target': 58}
{'time': 1.509484835986515, 'tau': 0.3088546346985337, 'owner': 35, 'action': -1, 'target': 35}
enter loop 1050
{'time': 1.7016216015633472, 'tau': 0.19213676557683218, 'owner': 35, 'action': -1, 'target': 35}
{'time': 1.5125294965985863, 'tau': 0.08031628418506713, 'owner': 74, 'action': -1, 'target': 74}
enter loop 1051
{'time': 1.6257046956570598, 'tau': 0.11317519905847354, 'owner': 74, 'action': -1, 'target': 74}
{'time': 1.512909190646202, 'tau': 0.23335093178663163, 'owner': 84, 'action': -1, 'target': 84}
enter loop 1052
{'time': 1.7191249095437757, 'tau': 0.2062157188975737, 'owner': 84, 'action': -1, 'target': 84}
{'time': 1.5163823870494202, 'tau': 0.2173708277511191, 'ow

{'time': 2.1354081462347545, 'tau': 0.171655131505911, 'owner': 81, 'action': -1, 'target': 81}
enter loop 1412
{'time': 2.284336611809868, 'tau': 0.14892846557511338, 'owner': 81, 'action': -1, 'target': 81}
{'time': 2.135720538462886, 'tau': 0.21953438847894252, 'owner': 28, 'action': -1, 'target': 28}
enter loop 1413
{'time': 2.2396142697336714, 'tau': 0.1038937312707855, 'owner': 28, 'action': -1, 'target': 28}
{'time': 2.1375306554727604, 'tau': 0.16021811499066702, 'owner': 46, 'action': -1, 'target': 46}
enter loop 1414
{'time': 2.284093161032007, 'tau': 0.14656250555924677, 'owner': 46, 'action': -1, 'target': 46}
{'time': 2.1440615106181142, 'tau': 0.10302015855443088, 'owner': 79, 'action': -1, 'target': 79}
enter loop 1415
{'time': 2.361905078379584, 'tau': 0.21784356776146968, 'owner': 79, 'action': -1, 'target': 79}
{'time': 2.1453843601785225, 'tau': 0.17853191371800495, 'owner': 59, 'action': -1, 'target': 59}
enter loop 1416
{'time': 2.365010008379224, 'tau': 0.21962564

enter loop 1827
{'time': 3.0025944543927405, 'tau': 0.2307165988395674, 'owner': 98, 'action': -1, 'target': 98}
{'time': 2.7725520150249174, 'tau': 0.062457641639232564, 'owner': 12, 'action': -1, 'target': 12}
enter loop 1828
{'time': 2.8342480279277664, 'tau': 0.06169601290284894, 'owner': 12, 'action': -1, 'target': 12}
{'time': 2.7737259881306717, 'tau': 0.19255948108240345, 'owner': 33, 'action': -1, 'target': 33}
enter loop 1829
{'time': 2.8690321329592323, 'tau': 0.09530614482856041, 'owner': 33, 'action': -1, 'target': 33}
{'time': 2.7744885716435586, 'tau': 0.24576670190543515, 'owner': 88, 'action': -1, 'target': 88}
enter loop 1830
{'time': 2.8944359869970557, 'tau': 0.11994741535349734, 'owner': 88, 'action': -1, 'target': 88}
{'time': 2.7761933267134755, 'tau': 0.014486943769627253, 'owner': 22, 'action': -1, 'target': 22}
enter loop 1831
{'time': 3.0678511162139364, 'tau': 0.29165778950046084, 'owner': 22, 'action': -1, 'target': 22}
{'time': 2.776450204824744, 'tau': 0.

...

enter loop 7243
{'time': 3.3727399233532864, 'tau': 0.1586533185229193, 'owner': 92, 'action': -1, 'target': 92}
{'time': 3.214094103334198, 'tau': 0.29125246294698226, 'owner': 64, 'action': -1, 'target': 64}
enter loop 7244
{'time': 3.310570204110142, 'tau': 0.09647610077594404, 'owner': 64, 'action': -1, 'target': 64}
{'time': 3.2143456363559, 'tau': 0.04097197732462428, 'owner': 43, 'action': -1, 'target': 43}
enter loop 7245
{'time': 3.5534044536552876, 'tau': 0.33905881729938736, 'owner': 43, 'action': -1, 'target': 43}
{'time': 3.214445402558638, 'tau': 0.2512632461283926, 'owner': 94, 'action': -1, 'target': 94}
enter loop 7246
{'time': 3.5165030878429797, 'tau': 0.3020576852843419, 'owner': 94, 'action': -1, 'target': 94}
{'time': 3.2144778582832627, 'tau': 0.14965658111304148, 'owner': 27, 'action': -1, 'target': 27}
enter loop 7247
{'time': 3.2915279706234775, 'tau': 0.07705011234021489, 'owner': 27, 'action': -1, 'target': 27}
{'time': 3.2145614769716784, 'tau': 0.090645884

enter loop 7743
{'time': 3.3063897979885204, 'tau': 0.00618853588170401, 'owner': 6, 'action': -1, 'target': 6}
{'time': 3.300431182818585, 'tau': 0.08421376811941926, 'owner': 46, 'action': -1, 'target': 46}
enter loop 7744
{'time': 3.4454958832465374, 'tau': 0.1450647004279524, 'owner': 46, 'action': -1, 'target': 46}
{'time': 3.3004904887174384, 'tau': 0.27921033588779365, 'owner': 55, 'action': -1, 'target': 55}
enter loop 7745
{'time': 3.3923834396998007, 'tau': 0.0918929509823624, 'owner': 55, 'action': -1, 'target': 55}
{'time': 3.3011165479371702, 'tau': 0.13969757773902852, 'owner': 13, 'action': -1, 'target': 13}
enter loop 7746
{'time': 3.4040051693510844, 'tau': 0.10288862141391418, 'owner': 13, 'action': -1, 'target': 13}
{'time': 3.3012844175571248, 'tau': 0.07242501246066733, 'owner': 71, 'action': -1, 'target': 71}
enter loop 7747
{'time': 3.587055481578588, 'tau': 0.28577106402146324, 'owner': 71, 'action': -1, 'target': 71}
{'time': 3.301372372213036, 'tau': 0.2186724

{'time': 3.3878419086443694, 'tau': 0.34537740609615364, 'owner': 9, 'action': -1, 'target': 9}
enter loop 8243
{'time': 3.4216803857862113, 'tau': 0.033838477141841794, 'owner': 9, 'action': -1, 'target': 9}
{'time': 3.388159589894928, 'tau': 0.26752985467177093, 'owner': 22, 'action': -1, 'target': 22}
enter loop 8244
{'time': 3.5615127662844768, 'tau': 0.17335317638954878, 'owner': 22, 'action': -1, 'target': 22}
{'time': 3.3882099946981716, 'tau': 0.28534546567719216, 'owner': 55, 'action': -1, 'target': 55}
enter loop 8245
{'time': 3.3913318106543806, 'tau': 0.0031218159562088066, 'owner': 55, 'action': -1, 'target': 55}
{'time': 3.3883364871436275, 'tau': 0.1251117001999111, 'owner': 70, 'action': -1, 'target': 70}
enter loop 8246
{'time': 3.683017957937683, 'tau': 0.2946814707940553, 'owner': 70, 'action': -1, 'target': 70}
{'time': 3.3884255289770375, 'tau': 0.3867696826255985, 'owner': 47, 'action': -1, 'target': 47}
enter loop 8247
{'time': 3.577220394626608, 'tau': 0.1887948

{'time': 3.473155859737701, 'tau': 0.18613067444221662, 'owner': 10, 'action': -1, 'target': 10}
enter loop 8743
{'time': 3.554224824053706, 'tau': 0.08106896431600542, 'owner': 10, 'action': -1, 'target': 10}
{'time': 3.4731584417594488, 'tau': 0.22954095774903185, 'owner': 19, 'action': -1, 'target': 19}
enter loop 8744
{'time': 3.76794851305282, 'tau': 0.2947900712933712, 'owner': 19, 'action': -1, 'target': 19}
{'time': 3.4734584999182836, 'tau': 0.07238134095498276, 'owner': 70, 'action': -1, 'target': 70}
enter loop 8745
{'time': 3.5853309318926003, 'tau': 0.11187243197431683, 'owner': 70, 'action': -1, 'target': 70}
{'time': 3.4734766688109, 'tau': 0.3007752457590333, 'owner': 33, 'action': -1, 'target': 33}
enter loop 8746
{'time': 3.653405958059074, 'tau': 0.17992928924817414, 'owner': 33, 'action': -1, 'target': 33}
{'time': 3.4738061340781763, 'tau': 0.05206410532272068, 'owner': 29, 'action': -1, 'target': 29}
enter loop 8747
{'time': 3.664638011469768, 'tau': 0.19083187739

KeyboardInterrupt: 

In [ ]:
print(len(timeline)/(males*t_max))